# NN in NLP 101
**W tym tygodniu brak zadania domowego!**

Do pracy na tekście w Pythonie najczęściej używane są biblioteki **NLTK**, **CoreNLP** lub **spaCy**. My dzisiaj będziemy używać tej ostatniej. Jest ona trochę szybsza w działaniu i jest zalecana do zastosowań produkcyjnych. W pierwszej kolejności zajmiemy się reprezentacją danych tekstowych. Stworzoną reprezentację użyjemy następnie w różnych problemach z dziedziny przetwarzania języka naturalnego.

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge spacy
!python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
import torch
import torch.nn as nn
import numpy as np

### I. Reprezentacja Danych

Aby użyć sieci neuronowych do analizy tekstu, musimy zareprezentować dane tekstowe w postaci wektorów liczb. Wektory mogą przedstawiać całe dokumenty, zdania, pojedyńcze słowa lub litery. My będziemy kodować słowa. Czasem zamiast słów używa się **lemmy**, czyli najbardziej podstawowej formy wyrazu (nieuwzględniającej odmiany, np. *pisać* zamiast *pisałem*, *mysz* zamiast *myszami*, itd.).

##### One Hot Encoding

Najbardziej podstawową reprezentacją jest one hot encoding. Wszystkie możliwe wyrazy stanowią słownik. Pojedyńcze słowo jest kodowane jako wektor o długości równej wielkości słownika. Wektor jest wypełniony zerami z jedynką na pozycji odpowiadającej danemu słowu.

Jak łatwo się domyślić, reprezentacja taka ma bardzo wysoki wymiar, gdyż słowniki mogą liczyć setki tysięcy słów. Reprezentacja ta jest rzadka i podobieństwo słów nie jest widoczne w reprezentacji.

Poniżej przykład reprezentacji słów w zdaniu „Ala ma kota i psa”:

| słowo | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ |
|-------|-------|-------|-------|-------|-------|
| Ala   | 1     | 0     | 0     | 0     | 0     |
| ma    | 0     | 1     | 0     | 0     | 0     |
| kota  | 0     | 0     | 1     | 0     | 0     |
| i     | 0     | 0     | 0     | 1     | 0     |
| psa   | 0     | 0     | 0     | 0     | 1     |

##### Word Embeddings

Lepszą reprezentacją do wielu problemów jest word embedding, który jest tworzony algorytmami uczenia nienadzorowanego takimi jak **word2vec** lub **GloVe**. Opierają się one o hipotezę dystrybucyjną, która mówi, że słowa występujące razem mają podobne znaczenie. SpaCy daje dostęp do wytrenowanych wektorów GloVe o długości 300 w języku angielskim.

In [ ]:
# Słownik zawiera 20000 wektorów GloVe o długości 300
print('Vocabulary dimensions:', nlp.vocab.vectors.shape)

# Gęste embeddingi mają ciekawe własności, np. król do królowej ma się tak jak mężczyzna do kobiety
print('king - man + woman = queen?')
a = nlp('king').vector - nlp('man').vector + nlp('woman').vector
b = nlp('queen').vector
print('cosine similarity:', np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# Pokrewne słowa mają duże podobieństwo między wektorami, a zupełnie niekojarzące się słowa - niskie podobieństwo
print('love and liver:', nlp('love').similarity(nlp('liver')))
print('love and heart:', nlp('love').similarity(nlp('heart')))

##### Embedding Layer

Warstwa tworzy embedding dostosowany do uczonego zadania. Na wejściu podaje się indeksy słów, a wyjściem są wektory embeddingów tych słów, które są uczone razem z pozostałą częścią sieci. Można o tej warstwie myśleć jak o warstwie liniowej do której podawane są wektory one-hot (używanie samych indeksów oszczędza miejsce, ale wynik jest równoważny).

Taka warstwa tworząca embedding może zostać zainicjalizowana przy pomocy wektorów GloVe. Przyspiesza to uczenie i dodaje dodatkową informację o słowach do modelu (GloVe trenowany był na innym korpusie tekstowym).

In [ ]:
# Tworzymy embedding layer odpowiednich wymiarów (wielkość słownika, docelowa wielkość wektora)
n_vocab, vocab_dim = nlp.vocab.vectors.shape
embedding_layer = nn.Embedding(n_vocab, vocab_dim)

# Wczytujemy wytrenowane wektory GloVe
embedding_layer.weight.data.copy_(torch.from_numpy(nlp.vocab.vectors.data))

# Sprawdźmy, czy nasza warstwa daje ten sam wynik, co zapisane w spaCy wektory
word = 'cucumber'

# Wczytaj wektor GloVe
spacy_vec = nlp(word).vector

# Znajdź indeks słowa w słowniku i oblicz wynik warstwy
key2row = nlp.vocab.vectors.key2row
vocab_id = nlp.vocab.strings[word]
row = key2row.get(vocab_id, None)
vocab_row = torch.tensor(row, dtype=torch.long)
embed_vec = embedding_layer(vocab_row)

# Porównaj wyniki
print(np.allclose(spacy_vec, embed_vec.detach().numpy()))

### II. Modelowanie Języka

Zadanie modelowania języka polega na przewidywaniu następnego słowa lub znaku w dokumencie. Proszę spróbować wykorzystać opisane wyżej embeddingi oraz poznaną na ostatnich zajęciach architekturę LSTM, aby stworzyć model języka. Jako dane można użyć na przykład korpusu [Penn Treebank](http://www.nltk.org/nltk_data/) lub [WikiText-2/WikiText-103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/). Można spróbować przewidywać jednocześnie następne i poprzednie słowo używając dwukierunkowego LSTM-a (argument `bidirectional` w klasie `nn.LSTM`). Uproszczony pipeline wygląda następująco:

Embedding > Batchowanie fragmentów tekstu > Warstwy LSTM lub GRU > Softmax na słowniku z predykcją kolejnego słowa

Można się zainspirować [tym artykułem](https://medium.com/the-artificial-impostor/notes-neural-language-model-with-pytorch-a8369ba80a5c).

In [ ]:
# CODE

### III. Analiza Sentymentu

Analiza sentymentu polega na klasyfikacji całego tekstu ze względu na emocje. W najbardziej podstawowej wersji jest to klasyfikacja wpisów na pozytywne i negatywne, czasem także neutralne. Przykładowymi zbiorami danych są [zbiór recenzji IMDB](http://ai.stanford.edu/~amaas/data/sentiment/) oraz [analiza tweetów](http://help.sentiment140.com/for-students/). Proszę spróbować wytrenować model na którymś ze zbiorów danych. Uproszczony pipeline:

Embedding > Batchowanie całych tekstów z odpowiednim paddingiem > Warstwy LSTM lub GRU > Softmax z wyborem klas

Przykładowe podejścia [tu](https://github.com/bentrevett/pytorch-sentiment-analysis), [tu](https://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948) i [tu](https://medium.com/@sonicboom8/sentiment-analysis-with-variable-length-sequences-in-pytorch-6241635ae130).

In [ ]:
# CODE